In [1]:
pip install pandas opencv-python scikit-learn tensorflow

In [2]:
pip install openpyxl

In [3]:
# Import necessary libraries
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from joblib import dump
import os

In [ ]:
# Function to load data from a CSV file
def load_data(file_path):
    # Adjust based on file format: CSV or Excel
    df = pd.read_csv(file_path)  # Use read_excel for .xlsx files if needed
    return df

In [ ]:
# Function to extract features using VGG16 from each frame of a video
def extract_features(frame, feature_extractor):
    # Resize the frame to 224x224 as required by VGG16
    frame = cv2.resize(frame, (224, 224))
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)

    # Extract features
    features = feature_extractor.predict(frame)
    return features.flatten()

In [ ]:
# Extract features for each video clip from the dataset
def extract_features_from_videos(df):
    # Instantiate VGG16 model once
    model = VGG16(weights='imagenet', include_top=False)
    feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    X = []  # Features
    y = []  # Labels

    for index, row in df.iterrows():
        video_path = row['Video Path']
        label = row['Labeling']  # 1 for A-rated, 0 for PG-13, -1 for U-rated

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error opening video file: {video_path}")
            continue

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Extract features every second or every few frames
            if frame_count % int(cap.get(cv2.CAP_PROP_FPS)) == 0:  # Sample 1 frame per second
                features = extract_features(frame, feature_extractor)
                X.append(features)
                y.append(label)

            frame_count += 1

        cap.release()
        print(f"Processed video: {video_path}")

    return np.array(X), np.array(y)

In [ ]:
# Train the classifier
def train_model(X, y):
    # Encode labels (A-rated as 1, PG-13 as 0, U-rated as -1)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train an SVM classifier
    model = SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)

    # Save the trained model and label encoder
    dump(model, '/content/drive/MyDrive/CAPSTONE/violence_classifier.joblib')
    dump(label_encoder, '/content/drive/MyDrive/CAPSTONE/label_encoder.joblib')

    accuracy = model.score(X_test, y_test) * 100
    print(f"Model trained with accuracy: {accuracy:.2f}%")
    return model

In [ ]:
# Main function to execute training
def main():
    csv_file_path = '/content/drive/MyDrive/CAPSTONE/cap_videos_2.csv'  # Replace with your actual CSV path
    df = load_data(csv_file_path)

    # Extract features from video clips
    print("Extracting features from videos...")
    X, y = extract_features_from_videos(df)

    # Train the model
    print("Training the model...")
    model = train_model(X, y)
    print("Model training completed and saved.")

# Run the main function
if __name__ == "__main__":
    main()

Extracting features from videos...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Processed video: /content/drive/MyDrive/CAPSTONE/Training dataset/P_7th_Sense_1.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Processed video: /content/drive/MyDrive/CAPSTONE/Training dataset/P_7th_Sense_2.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Processed video: /content/drive/MyDrive/CAPSTONE/Training dataset/P_7th_Sense_3.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━

In [4]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from joblib import load
from collections import Counter

In [5]:
# Load the model and label encoder
def load_trained_model(model_path, label_encoder_path):
    model = load(model_path)
    label_encoder = load(label_encoder_path)
    return model, label_encoder

In [6]:
# Function to extract features for prediction
def extract_features_for_prediction(frame, feature_extractor):
    frame = cv2.resize(frame, (224, 224))
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)
    features = feature_extractor.predict(frame)
    return features.flatten()

In [7]:
# Function to classify the entire video
def classify_video(input_video_path, model, label_encoder):
    # Prepare the VGG16 feature extractor
    base_model = VGG16(weights='imagenet', include_top=False)
    feature_extractor = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

    cap = cv2.VideoCapture(input_video_path)
    frame_count = 0
    predictions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract features every second or every few frames
        if frame_count % int(cap.get(cv2.CAP_PROP_FPS)) == 0:  # Sample 1 frame per second
            features = extract_features_for_prediction(frame, feature_extractor)
            features = np.expand_dims(features, axis=0)
            prediction = model.predict(features)
            predicted_label = label_encoder.inverse_transform(prediction)[0]
            predictions.append(predicted_label)

        frame_count += 1

    cap.release()

    # Determine the overall rating based on predictions
    prediction_counts = Counter(predictions)
    if 1 in prediction_counts:
        return "A-rated"
    elif 0 in prediction_counts:
        return "PG-13-rated"
    else:
        return "U-rated"

In [9]:
# Example usage:
def main():
    # Paths to model and encoder
    model_path = '/content/drive/MyDrive/CAPSTONE/violence_classifier.joblib'
    label_encoder_path = '/content/drive/MyDrive/CAPSTONE/label_encoder.joblib'

    # Load the trained model and encoder
    model, label_encoder = load_trained_model(model_path, label_encoder_path)

    # Classify a new input video
    input_video_path = '/content/drive/MyDrive/CAPSTONE/Sample dataset/Salaar_3.mp4'  # Replace with the path to the video you want to classify
    classification = classify_video(input_video_path, model, label_encoder)
    print(f"The video is classified as: {classification}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_19']
Received: inputs=Tensor(shape=(1, 224, 224, 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━